In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('D:\data engine\数据分析训练营-结营考试\ProjectB/订单表.csv',encoding='gbk')

In [3]:
data1= data.sort_values('客户ID',ascending=True)
data1

,订单日期,年份,订单数量,产品ID,产品型号名称,产品型号名称.1,产品类别,单价,销售金额,产品成本,利润,客户ID,交易类型,销售区域ID,销售大区,国家,区域
24925,2015/7/22,2015,1,485,Fender Set - Mountain,挡泥板,配件,21.9800,21.9800,8.2205,13.7595,11000,1,9,中国澳门,中国澳门,大中华区
32933,2015/11/4,2015,1,530,Touring Tire Tube,车胎和内胎,配件,4.9900,4.9900,1.8663,3.1237,11000,1,9,中国澳门,中国澳门,大中华区
25875,2015/7/22,2015,1,353,Mountain-200,山地自行车,自行车,2319.9900,2319.9900,1265.6195,1054.3705,11000,1,9,中国澳门,中国澳门,大中华区
34708,2013/7/22,2013,1,344,Mountain-100,山地自行车,自行车,3399.9900,3399.9900,1912.1544,1487.8356,11000,2,9,中国澳门,中国澳门,大中华区
25639,2015/11/4,2015,1,214,Sport-100,头盔,配件,34.9900,34.9900,13.0863,21.9037,11000,1,9,中国澳门,中国澳门,大中华区
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37135,2016/1/18,2016,1,477,Water Bottle,水壶和水壶架,配件,4.9900,4.9900,1.8663,3.1237,29480,2,10,中国台湾,中国台湾,大中华区
36875,2016/1/18,2016,1,217,Sport-100,头盔,配件,34.9900,34.9900,13.0863,21.9037,29480,2,10,中国台湾,中国台湾,大中华区
15205,2014/2/13,2014,1,349,Mountain-100,山地自行车,自行车,3374.9900,3374.9900,1898.0944,1476.8956,29481,1,8,中国香港,中国香港,大中华区
49016,2015/3/22,2015,1,358,Mountain-200,山地自行车,自行车,2049.0982,2049.0982,1105.8100,943.2882,29482,3,7,新加坡,新加坡,新加坡


In [4]:
# 采用efficient_apriori工具包
def rule1():# 得到一维数组orders_series，并且将Transaction作为index, value为Item取值
    from efficient_apriori import apriori
    orders_series = data1.set_index('客户ID')['产品型号名称']
    # 将数据集进行格式转换
    客户ID = []
    temp_index = 0
    for i, v in orders_series.items():
        if i != temp_index:
            temp_set = set()
            temp_index = i
            temp_set.add(v)
            客户ID.append(temp_set)
        else:
            temp_set.add(v)
    # 挖掘频繁项集和频繁规则
    itemsets,rules = apriori(客户ID, min_support=0.05, min_confidence=0.2)
    print('频繁项集：', itemsets)
    print('关联规则：', rules)

In [5]:
rule1()

频繁项集： {1: {('Fender Set - Mountain',): 2110, ('Touring Tire Tube',): 1411, ('Touring-1000',): 1238, ('Short-Sleeve Classic Jersey',): 1562, ('Sport-100',): 5960, ('Mountain-200',): 3011, ('Mountain Bottle Cage',): 2004, ('Water Bottle',): 4073, ('Road Bottle Cage',): 1700, ('Road-750',): 1443, ('Cycling Cap',): 2132, ('Patch kit',): 2950, ('HL Mountain Tire',): 1396, ('Mountain Tire Tube',): 2960, ('Road Tire Tube',): 2226, ('Half-Finger Gloves',): 1376, ("Women's Mountain Shorts",): 1019, ('Long-Sleeve Logo Jersey',): 1684, ('LL Road Tire',): 1030, ('ML Mountain Tire',): 1161, ('Road-150',): 1551, ('Road-250',): 1731, ('Road-350-W',): 929, ('Road-550-W',): 1313}, 2: {('Mountain-200', 'Sport-100'): 1127, ('Mountain Bottle Cage', 'Water Bottle'): 1704, ('Road Bottle Cage', 'Water Bottle'): 1522, ('Sport-100', 'Water Bottle'): 1324, ('HL Mountain Tire', 'Mountain Tire Tube'): 1001, ('Mountain Tire Tube', 'Patch kit'): 954, ('Mountain Tire Tube', 'Sport-100'): 1449, ('Road Tire Tube', 'Sp

In [6]:
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

In [10]:
# 采用mlxtend.frequent_patterns工具包
def rule2():
    pd.options.display.max_columns=100
    hot_encoded_df=data.groupby(['客户ID','产品型号名称'])['产品型号名称'].count().unstack().reset_index().fillna(0).set_index('客户ID')
    hot_encoded_df = hot_encoded_df.applymap(encode_units)
    from mlxtend.frequent_patterns import apriori
    from mlxtend.frequent_patterns import association_rules
    frequent_itemsets = apriori(hot_encoded_df, min_support=0.03, use_colnames=True)
    frequent_itemsets = frequent_itemsets.sort_values(by="support", ascending=False)
    rules = association_rules(frequent_itemsets, metric="lift", min_threshold=0.5)
    rules = rules.sort_values(by="lift", ascending=False)
    print("频繁项集：", frequent_itemsets)
    print("关联规则：", rules[ (rules['lift'] >= 1) & (rules['confidence'] >= 0.5) ])   

In [11]:
rule2()

频繁项集：      support                                           itemsets
27  0.322441                                        (Sport-100)
31  0.220353                                     (Water Bottle)
15  0.162898                                     (Mountain-200)
14  0.160138                               (Mountain Tire Tube)
16  0.159597                                        (Patch kit)
..       ...                                                ...
41  0.030567                     (Road Tire Tube, LL Road Tire)
54  0.030405                       (Road-750, Road Bottle Cage)
17  0.030242                                     (Racing Socks)
1   0.030134                                     (Classic Vest)
62  0.030134  (Sport-100, HL Mountain Tire, Mountain Tire Tube)

[65 rows x 2 columns]
关联规则：                                antecedents             consequents  \
20                     (Touring Tire Tube)          (Touring Tire)   
21                          (Touring Tire)     (Touring T